In [1]:
import sqlite3, os
import pandas as pd
import matplotlib.pyplot as plt

DB_PATH = "data/reco.db"  
con = sqlite3.connect(DB_PATH)
print("Using DB:", DB_PATH)


Using DB: data/reco.db


In [2]:
tables = ["ml_movies", "ml_ratings", "ml_tags", "imdb_title_basics", "imdb_title_ratings"]
sizes = []
for t in tables:
    try:
        n = pd.read_sql(f"SELECT COUNT(*) AS n FROM {t}", con)["n"].iloc[0]
        sizes.append((t, n))
    except Exception:
        sizes.append((t, None))
pd.DataFrame(sizes, columns=["table","rows"])


,table,rows
0,ml_movies,54556
1,ml_ratings,40000526
2,ml_tags,931128
3,imdb_title_basics,24054768
4,imdb_title_ratings,3264212


In [3]:
ratings = pd.read_sql("SELECT userId, movieId, rating FROM ml_ratings", con)
users  = ratings["userId"].nunique()
movies = ratings["movieId"].nunique()
mean, std = ratings["rating"].mean(), ratings["rating"].std()
print("users:", users, "movies:", movies, "ratings:", len(ratings))
print("rating mean ± std:", round(mean,3), "±", round(std,3))

u_cnt = ratings.groupby("userId").size()
i_cnt = ratings.groupby("movieId").size()
print("median ratings/user:", int(u_cnt.median()), " p90:", int(u_cnt.quantile(0.9)))
print("median ratings/movie:", int(i_cnt.median()), " p90:", int(i_cnt.quantile(0.9)))

print("users with 1 rating:", int((u_cnt==1).sum()))
print("movies with 1 rating:", int((i_cnt==1).sum()))


users: 138493 movies: 26744 ratings: 40000526
rating mean ± std: 3.526 ± 1.052
median ratings/user: 136  p90: 668
median ratings/movie: 36  p90: 2611
users with 1 rating: 0
movies with 1 rating: 0


In [4]:
movies_df = pd.read_sql("SELECT movieId, title, genres FROM ml_movies", con)
genres = movies_df["genres"].str.get_dummies(sep="|").sum().sort_values(ascending=False)
genres.head(20).to_frame("count")


,count
Drama,26688
Comedy,16748
Thriller,8356
Romance,8254
Action,7040
Crime,5878
Horror,5222
Documentary,4942
Adventure,4658
Sci-Fi,3486


In [5]:
coverage = 100.0 * ratings["movieId"].isin(movies_df["movieId"]).mean()
print("ratings_covered_%:", round(coverage,2))
con.close()


ratings_covered_%: 100.0
